# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

**Import**

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

**Constants**

In [ ]:
BUDGET = 10**10
INCOME_PER_BARREL = 450000
POINTS = 500
PREDICT_POINTS = 200
LOSE_PROBABILITY =  0.025
VOLUME=BUDGET/(INCOME_PER_BARREL*PREDICT_POINTS)

**Read data**

In [ ]:
df0=pd.read_csv('data/geo_data_0.csv')
df0.name='1 location'
df1=pd.read_csv('data/geo_data_1.csv')
df1.name='2 location'
df2=pd.read_csv('data/geo_data_2.csv')
df2.name='3 location'
dfs=[df0,df1,df2]
df0.head()

**Functions**

In [ ]:
def info(df):
    print(df.name)
    print('shape:', df.shape)
    print('Na:',df.isna().sum().sum())
    print('Duplicates:',df.duplicated().sum())
    print('Describe:')
    print(df.describe())


In [ ]:
def hists(df):
    for column in df.columns[1:]:
        title = 'Распределение значений столбца ' + column 
        fig = px.histogram(df,
                           x = column,
                           marginal = 'box',
                           nbins=30,
                           width=600, height=300,
                           title = title)

        fig.show()

In [ ]:
def split(df,test_size=0.25,random_state=12345,stratify=None):
    features = df.drop(['product','id'],axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=test_size, random_state=random_state,stratify = stratify)
    return features_train, features_valid, target_train, target_valid

In [ ]:
def learn_model(features_train, features_valid, target_train, target_valid):
    scaler = StandardScaler()
    scaler.fit(features_train) 
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse ** 0.5
    average = sum(predicted_valid) / len(predicted_valid)
    predicted_valid = pd.Series(predicted_valid)
    predicted_valid.index=target_valid.index
    return predicted_valid,rmse,average

In [ ]:
def revenue(target, prediction,count):
    volume_predict = prediction.sort_values(ascending=False)[:count]
    volume_real = target[volume_predict.index]
    profit = volume_real.sum()*INCOME_PER_BARREL - BUDGET
    return profit

In [ ]:
state=np.random.RandomState(12345)
def profit(target,prediction):
    profit_values=[]
    for i in range(1000):
        target_subsample = target.sample(n=POINTS, replace=True, random_state=state)
        predictions_subsample = prediction[target_subsample.index]
        profit_values.append(revenue(target_subsample,predictions_subsample,PREDICT_POINTS))
    profit_values = pd.Series(profit_values)
    confidence_interval = (profit_values.quantile(0.025),profit_values.quantile(0.975))
    mean = profit_values.mean()
    risk=(profit_values<0).mean()
    return confidence_interval,mean,risk

**Data info**

In [ ]:
for df in dfs:
    info(df)
    print('-----------------------------------------------------')

In [ ]:
for df in dfs:
    print('--------------------------------------------')
    print(df.name)
    hists(df)

Предварительный анализ данных выявил отсутствие дубликатов и пропущенных данных. В основном данные по параметрам f0,f1,f2 близки к нормальному распределению. Имеются различия в масштабе данных, поэтому при обучении модели решено применить масштабирование признаков. Так же в некоторых случаях наблюдаются выбросы, которые решено было оставить, т.к. без эксперта невозможно определить важность этих данных.

## Обучение и проверка модели

In [ ]:
for df in dfs:
    features_train, features_valid, target_train, target_valid = split(df)
    predicted_valid,rmse,average = learn_model(features_train, features_valid, target_train, target_valid)
    confidence_interval,mean,risk = profit(target_valid, predicted_valid)
    print('--------------------------------------')
    print(df.name)
    print('RMSE:',rmse,'\nAverage:',average)

Наименьшее значение среднеквадратической ошибки приходится на 2 участок, наибольшее среднее значение объемов нефи - на 3 участок. 

## Подготовка к расчёту прибыли

In [ ]:
print('Достаточный объём сырья для безубыточной разработки новой скважины:',np.ceil(VOLUME))

In [ ]:
for df in dfs:
    print('--------------------')
    print(df.name)
    for q in np.arange(0.6,0.9,0.01):
        print(q,df['product'].quantile(q))

Достаточный объём сырья для безубыточной разработки новой скважины составил 112 тысяч баррелей(с округлением вверх). По предыдущим значениям моделей средние показатели моделей оказались ниже минимального порога. Однако по данным описательной статистики видно, что во всех регионах наблюдаются значения выше указанного (35% для 1 региона, 16% для 2 региона, 37% для 3).

## Расчёт прибыли и рисков 

In [ ]:
for df in dfs:
    features_train, features_valid, target_train, target_valid = split(df)
    predicted_valid,rmse,average = learn_model(features_train, features_valid, target_train, target_valid)
    confidence_interval,mean,risk = profit(target_valid, predicted_valid)
    print('--------------------------------------')
    print(df.name)
    print('Среднее значение прибыли для региона:',mean)
    print('95%-й доверительный интервал для значения прибыли:',confidence_interval)
    print('Риск убытков:',risk)

Риск убытков в 3 регионе оказался больше 2.5%, поэтому его отметаем. Из оставшихся регионов 2 регион показал наибольшую среднее значение рассчитанной прибыли, а так же наименьшие риски убытков. Таким образом, не смотря на то, что при проверки модели 2 регион показал самые плохие результаты, использование бутстрапа выявило наибольшую прибыльность именно в этом регионе.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован